In [101]:
import numpy as np

###############################################################################
##Defining the necessities
#Defining (n, k)
num_channels = 2
num_messages = 4
num_bits = 2
################################################################################
##Generating the training data, one hot vectors of width num_messages
num_samples = 10000
training_data = np.zeros([num_samples, num_messages])

file = open("training_data-a.npy", 'wb')
file1 = open("training_data-b.npy", 'wb')

for iter in range(num_samples):

    training_data[iter, np.random.randint(num_messages)] = 1
np.save(file, training_data)
print(training_data)

training_data = np.zeros([num_samples, num_messages])

for iter in range(num_samples):
    training_data[iter, np.random.randint(num_messages)] = 1

np.save(file1, training_data)
print(training_data)
file.close()
file1.close()
###############################################################################
##Generating the test data, one hot vectors of width num_messages
num_samples = 5000
test_data = np.zeros([num_samples, num_messages])

file = open("test_data-a.npy", 'wb')
file1 = open("test_data-b.npy", 'wb')

for iter in range(num_samples):
    test_data[iter, np.random.randint(num_messages)] = 1

np.save(file, test_data)
print(test_data)

test_data = np.zeros([num_samples, num_messages])

for iter in range(num_samples):
    test_data[iter, np.random.randint(num_messages)] = 1

np.save(file1, test_data)

file.close()
file1.close()


[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [102]:
file_train = open("training_data-a.npy", "rb")
file_train1 = open("training_data-b.npy", "rb")
file_test = open("test_data-a.npy", "rb")
file_test1 = open("test_data-b.npy", "rb")

training_data_a = np.load(file_train)
test_data_a = np.load(file_test)
training_data_b = np.load(file_train1)
test_data_b = np.load(file_test1)

training_data = np.concatenate((training_data_a, training_data_b), axis = 1)
test_data  = np.concatenate((test_data_a, test_data_b), axis = 1)

file_train.close()
file_test.close()
file_train1.close()
file_test1.close()

print(training_data)
print(test_data.shape)

[[1. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 1.]
 ...
 [1. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
(5000, 8)


In [103]:
import keras
from keras.layers import Input, Dense
from keras.optimizers import SGD, Adam
from keras.models import Model
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tensorflow as tf

In [104]:
learning_rate = 0.001
msg_len = 4
input_dim = 2*msg_len
channel_num = 2
encoding_dim = 2*channel_num
midlayer_dim = int((msg_len+channel_num)/2)+1
print ("midlayer_dim :", midlayer_dim )
energy_per_bit1 = 5
energy_per_bit2 = 5

midlayer_dim : 4


In [115]:
input_msg = Input(shape = (input_dim, ))
print("Input shape : ", input_dim)
# input_msg2 = Input(shape = (msg_len, ))


lambda_l1_1 = keras.layers.Lambda(lambda x : x[0:msg_len, :])(input_msg)
encoded_l1_1 = Dense(msg_len, activation = 'relu')(lambda_l1_1)
encoded_l1_2 = Dense(channel_num, activation = 'linear')(encoded_l1_1)
encoded_l1_3 = keras.layers.BatchNormalization(axis=1)(encoded_l1_2)

lambda_l2_1 = keras.layers.Lambda(lambda x : x[msg_len:2*msg_len, :])(input_msg)
encoded_l2_1 = Dense(msg_len, activation = 'relu')(lambda_l2_1)
encoded_l2_2 = Dense(channel_num, activation = 'linear')(encoded_l2_1)
encoded_l2_3 = keras.layers.BatchNormalization(axis=1)(encoded_l2_2)

added_layer = keras.layers.Add()([encoded_l1_3, encoded_l2_3])
encoded_l1_4 = keras.layers.GaussianNoise(np.sqrt(channel_num/(2*msg_len*energy_per_bit1)))(added_layer)
encoded_l2_4 = keras.layers.GaussianNoise(np.sqrt(channel_num/(2*msg_len*energy_per_bit2)))(added_layer)

decoded_l1_1 = Dense(midlayer_dim, activation = 'relu')(encoded_l1_4)
decoded_l2_1 = Dense(midlayer_dim, activation = 'relu')(encoded_l2_4)

decoded_l1_2 = Dense(midlayer_dim, activation = 'softmax')(decoded_l1_1)
decoded_l2_2 = Dense(midlayer_dim, activation = 'softmax')(decoded_l2_1)

concat_layer = keras.layers.concatenate([decoded_l1_2, decoded_l2_2])

decoded_out = Dense(input_dim)(concat_layer)


Input shape :  8


In [116]:
adam = Adam(learning_rate)
autoencoder = Model(inputs = input_msg, outputs = decoded_out)
autoencoder.compile(optimizer = adam, loss='categorical_crossentropy')

In [117]:
encoder1 = Model(input_msg, encoded_l1_3)
encoder2 = Model(input_msg, encoded_l2_3)

In [124]:
print(training_data.shape)
print(keras.backend.shape(input_msg))
autoencoder.fit(training_data,training_data, epochs=100, batch_size=4, shuffle=True,validation_data=(test_data, test_data))

(10000, 8)
Tensor("Shape_7:0", shape=(2,), dtype=int32)
Train on 10000 samples, validate on 5000 samples
Epoch 1/100


InvalidArgumentError: Incompatible shapes: [4,2] vs. [0,2]
	 [[{{node training_1/Adam/gradients/add_8_1/add_grad/BroadcastGradientArgs}} = BroadcastGradientArgs[T=DT_INT32, _class=["loc:@training_1/Adam/gradients/batch_normalization_52/cond/Merge_grad/cond_grad"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](training_1/Adam/gradients/add_8_1/add_grad/Shape, training_1/Adam/gradients/add_8_1/add_grad/Shape_1)]]